Below I show that for many classes of neuron models using a parallel Rheobase algorithm adds a significant speed up to model evaluation. It is important to consider that determinig rheobase usually involves between 10-35 model simulations at different current injection strengths. Where is the remaining error score calculations only invole at most one simulation each. Consider an optimization problem where there are only eight error scores to calculate, and four of the eight errors are for non spiking simulations of less than 500ms, a rheobase simulation is for 1200ms, and it involves both non-spiking and multi-spiking behavior. Multi-spiking models take longer to simulate than non spiking models for technical reasons. In this particular instance where finding the unique rheobase value for each different gene is important, that rheobase determination is the biggest computational bottleneck.

It is also important to consider that the speed of model simulation is often determined by the exact parameterization of the model in question. During optimization parameterization frequently changes. Multi-spiking models that are slower to evaluate are frequently sampled by a genetic algorithm optimizer. 

In [4]:
from neuronunit.tests.fi import RheobaseTestP
import pickle
tests = pickle.load(open("processed_multicellular_constraints.p","rb"))
nu_tests = tests['Hippocampus CA1 pyramidal cell'].tests
from bluepyopt.ephys.models import ReducedCellModel
import numpy
from neuronunit.optimisation.model_parameters import MODEL_PARAMS
from bluepyopt.ephys.models import ReducedCellModel
import bluepyopt.ephys as ephys
from neuronunit.optimisation.brian_glif_model_parameters import transcribe_units
import sciunit, neuronunit, quantities
from neuronunit.tests.dm import *
import time
from neuronunit.tests import dm #this is me importing the Druckman tests
from neuronunit.tests import RheobaseTestP, fi, RheobaseTest 
from neuronunit.models.reduced import ReducedModel
from neuronunit.optimisation.data_transport_container import DataTC


In [11]:

tests = pickle.load(open("processed_multicellular_constraints.p","rb"))
nu_tests = tests['Hippocampus CA1 pyramidal cell'].tests
#rp = RheobaseTestP(observation = nu_tests[0].observation)

from neuronunit.optimisation.brian_glif_model_parameters import MODEL_PARAMS as MODEL_PARAMS_depreciated

IZHI_cell = ephys.models.ReducedCellModel(
        name='NEURON',
        params=MODEL_PARAMS_depreciated["NEURON_IZHI"],backend="NEURON") 
dtc = DataTC()
dtc.attrs = IZHI_cell.attrs
dtc.ampl=0
dtc.attrs_raw = {'C':89.7960714285714, 'a':0.01, 'b':15, 'c':-60, 'd':10,\
         'k':1.6, 'vPeak':(86.364525297619-65.2261863636364),\
         'vr':-65.2261863636364, 'vt':-50}


dtc.attrs = transcribe_units(dtc.attrs_raw)
IZHI_cell.attrs = dtc.attrs
start0 = time.time()
rtp = RheobaseTestP(observation = nu_tests[0].observation)
pred0 = rtp.generate_prediction(IZHI_cell)
end0 = time.time()
#print(IZHI_cell.attrs)

start1 = time.time()
rt = fi.RheobaseTest(observation = nu_tests[0].observation)
pred1 = rt.generate_prediction(IZHI_cell)
end1 = time.time()

print('parallel Rheobase search time NEURON ', end0-start0)
print('serial Rheobase search time NEURON ',end1-start1)


print('speed up {0}'.format((end1-start1)/(end0-start0)))

time taken on block 4.555711984634399 
time taken on block 18.55247473716736 
parallel Rheobase search time NEURON  4.831164836883545
serial Rheobase search time NEURON  18.69422459602356
speed up 3.8695066774170974


In [10]:
explore_ranges = {'E_Na' : (40,70), 'E_K': (-90.0,-75.0)}

attrs = { 'g_K' : 36.0, 'g_Na' : 120.0, 'g_L' : 0.3, \
         'C_m' : 1.0, 'E_L' : -54.387, 'E_K' : -77.0, 'E_Na' : 50.0, 'vr':-65.0 } 

cond_model = ephys.models.ReducedCellModel(
        name='HH',
        params=MODEL_PARAMS_depreciated["HH"],backend="HH") 


#model = ReducedModel(LEMS_MODEL_PATH,name = str('vanilla'),backend = ('HH'))
cond_model.attrs = attrs
cond_model.attrs = attrs
iparams = {}
iparams['injected_square_current'] = {}
iparams['injected_square_current']['amplitude'] = 1.98156805*pq.pA
iparams['injected_square_current']['amplitude'] = 2.98156805*pq.pA

DELAY = 100.0*pq.ms
DURATION = 1000.0*pq.ms
iparams['injected_square_current']['delay'] = DELAY
iparams['injected_square_current']['duration'] = int(DURATION)
bf = time.time()
cond_model.inject_square_current(iparams)
vm = cond_model.get_membrane_potential()
af = time.time()

#volts = [v[0] for v in vm ]
#print(len(vm[0]),len(vm.times))


bfs = time.time()
#model = ReducedModel(LEMS_MODEL_PATH, name= str('vanilla'), backend=('HH', {'DTC':dtc}))
#model._backend.cell_name = str('vanilla')
#rt = RheobaseTestP(obs_frame['Dentate gyrus basket cell']['Rheobase'])
pred0 = rt.generate_prediction(cond_model)
afs = time.time()
print('elapsed serial: ',afs-bfs)


bfp = time.time()
#model = ReducedModel(LEMS_MODEL_PATH, name= str('vanilla'), backend=('HH', {'DTC':dtc}))
#model._backend.cell_name = str('vanilla')

#rt = RheobaseTest(obs_frame['Dentate gyrus basket cell']['Rheobase'])
pred0 = rtp.generate_prediction(cond_model)
afp = time.time()
print('elapsed parallel: ',afp-bfp)
print('speed up parallel: ',(afs-bfs)/(afp-bfp))


time taken on block 0.7909517288208008 
elapsed serial:  0.7914438247680664
time taken on block 0.2581171989440918 
elapsed parallel:  0.2590057849884033
speed up parallel:  3.055699411515084
